## Data preparation for encoding/decoding models

For this example, we will use the data from Haxby et al., 2001., which are shared via OpenNeuro:

https://openneuro.org/datasets/ds000105/versions/3.0.0

The data are formatted according to the BIDS standard: https://bids-specification.readthedocs.io/en/stable/index.html

First, import required dependencies. You can install these using `pip install -r requirements.txt` from the main repo directory.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import nilearn
from nilearn import datasets
from nilearn.maskers import NiftiMasker
from nilearn.maskers import MultiNiftiMapsMasker
import h5py
import numpy as np
import nibabel as nib
import datalad.api as dl
import pandas as pd
import tqdm
from utils import (
    get_subject_common_brain_mask,
    get_group_common_mask,
    get_subject_runs,
    get_layouts,
)

#### Get the data using datalad

We will use a tool called [Datalad](https://www.datalad.org/) to obtain the data from openneuro.

We will download the raw data, as well as the processed data (using [fMRIPrep](https://fmriprep.org/en/stable/).  Note that downloading these derivative data can take quite a while depending on the speed of one's connection.

In [ ]:
data_dir = '/Users/poldrack/data_unsynced/ds000105'
assert os.path.exists(data_dir), 'Data directory not found: %s' % data_dir
fmriprep_dir = os.path.join(data_dir, 'derivatives', 'fmriprep')

output_dir = os.path.join(data_dir, 'derivatives', 'cleaned')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# get the raw data
ds = dl.clone(
    path=data_dir,
    source='https://github.com/OpenNeuroDatasets/ds000105.git',
)
dl.get(dataset=data_dir, recursive=True)

get_fmriprep = False  # set to false after downloading fmriprep once

# get the preprocessed derivatives - this takes some time!
if get_fmriprep:
    dl.clone(
        path=fmriprep_dir,
        source='https://github.com/OpenNeuroDerivatives/ds000105-fmriprep.git',
    )
    dl.get(dataset=fmriprep_dir, recursive=True)

We also need to grab the data from nilearn, which contains the visual cortical masks used in the original study.

In [ ]:

# First, we fetch single subject specific data with haxby datasets: to have
# anatomical image, EPI images and masks images
haxby_dataset = datasets.fetch_haxby()
haxby_datadir = os.path.dirname(haxby_dataset.mask)

### Query the dataset using PyBIDS

Because the dataset is organized using the BIDS standard, we can use the [PyBIDS](https://bids-standard.github.io/pybids/) tool to query the dataset and obtain useful metadata.


In [ ]:
# load the dataset using pybids and get runs for each subject


layout, deriv_layout = get_layouts(data_dir, fmriprep_dir)

### create 3mm resolution version of the data



In [ ]:
boldfiles = deriv_layout.get(
    desc='preproc',
    space='MNI152NLin2009cAsym',
    suffix='bold',
    extension='nii.gz',
    res=2,
    return_type='file',
)
print(f'found {len(boldfiles)} bold files')

target_affine = np.diag((3, 3, 3))

for boldfile in tqdm.tqdm(boldfiles, desc='Processing items'):
    output = boldfile.replace('res-2', 'res-3')
    assert output != boldfile
    if not os.path.exists(output):
        img = nilearn.image.resample_img(
            boldfile,
            target_affine=target_affine,
            copy_header=True,
            force_resample=True,
        )
        img.to_filename(output)
    # create boldref as well
    boldref = boldfile.replace('desc-preproc_bold.nii.gz', 'boldref.nii.gz')
    output_boldref = boldref.replace('res-2', 'res-3')
    assert output_boldref != boldref
    if not os.path.exists(output_boldref):
        img = nilearn.image.resample_img(
            boldref,
            target_affine=target_affine,
            copy_header=True,
            force_resample=True,
        )
        img.to_filename(output_boldref)
    # create mask
    maskfile = boldfile.replace(
        'desc-preproc_bold.nii.gz', 'desc-brain_mask.nii.gz'
    )
    output_maskfile = maskfile.replace('res-2', 'res-3')
    assert output_maskfile != maskfile
    if not os.path.exists(maskfile):
        img = nilearn.image.resample_img(
            maskfile,
            target_affine=target_affine,
            interpolation='nearest',
            copy_header=True,
            force_resample=True,
        )
        img.to_filename(output_maskfile)

In [ ]:
boldfile

### Create common mask for each subject

Each run will have slightly different voxels included in its brain mask, but we want to have a common mask across all runs, so we will generate a mask that includes the intersection of masks across all of the individual subs/runs.

In [ ]:
group_mask = get_group_common_mask(layout, res=3)

### Confound regression

Use the outputs from fMRIPrep to generate a denoised version of the data.



In [ ]:
def run_confound_regression(
    layout, deriv_layout, data_dir, res=3, overwrite=False
):
    cleaned_images = {}

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        cleaned_images[subject] = {}
        runs = get_subject_runs(subject, data_dir)
        print(f'Subject {subject} has {len(runs)} runs')
        mask_img = get_subject_common_brain_mask(subject, data_dir, res=res)
        # mask_img = resample_img(mask_img, target_affine=np.diag((3, 3, 3)),
        #                        interpolation='nearest',
        #                        copy_header=True, force_resample=True)
        t_r = None
        for run in runs:
            preproc_file = deriv_layout.get(
                subject=subject,
                run=run,
                res=res,
                desc='preproc',
                space='MNI152NLin2009cAsym',
                suffix='bold',
                extension='nii.gz',
                return_type='file',
            )
            cleaned_img_file = preproc_file[0].replace('preproc', 'cleaned')
            if t_r is None:
                t_r = deriv_layout.get_metadata(preproc_file[0])[
                    'RepetitionTime'
                ]
            else:
                assert (
                    t_r
                    == deriv_layout.get_metadata(preproc_file[0])[
                        'RepetitionTime'
                    ]
                )
            assert t_r is not None
            if os.path.exists(cleaned_img_file) and not overwrite:
                # print(f"Using existing cleaned file for subject {subject} run {run}")
                cleaned_img = nib.load(cleaned_img_file)
                cleaned_images[subject][run] = (cleaned_img_file, cleaned_img)
                continue
            preproc_img = nib.load(preproc_file[0])
            assert (
                len(preproc_file) == 1
            ), f'Found {len(preproc_file)} preproc files for subject {subject} run {run}'
            confound_file = deriv_layout.get(
                subject=subject,
                run=run,
                desc='confounds',
                suffix='timeseries',
                extension='tsv',
                return_type='file',
            )
            assert (
                len(confound_file) == 1
            ), f'Found {len(confound_file)} confound files for subject {subject} run {run}'
            confounds = pd.read_csv(confound_file[0], sep='\t').bfill()
            # need to include cosine with acompcor
            confound_prefixes = ['trans_', 'rot_', 'a_comp_cor_', 'cosine']
            confound_cols = [
                c
                for c in list(confounds.columns)
                if any([c.startswith(p) for p in confound_prefixes])
            ]
            confounds_selected = confounds[confound_cols]
            cleaned_img = nilearn.image.clean_img(
                preproc_img,
                confounds=confounds_selected,
                t_r=t_r,
                mask_img=mask_img,
            )
            assert cleaned_img_file != preproc_file[0]
            cleaned_img.to_filename(os.path.join(cleaned_img_file))
            cleaned_images[subject][run] = (cleaned_img_file, cleaned_img)
    return cleaned_images, t_r


# refresh the layouts to detect the new res-3 files
layout, deriv_layout = get_layouts(data_dir, fmriprep_dir)

cleaned_images, t_r = run_confound_regression(
    layout, deriv_layout, data_dir, overwrite=True, res=3
)

cleaned_images, t_r = run_confound_regression(
    layout, deriv_layout, data_dir, overwrite=True, res=2
)

### select task block timepoints

drop the first two TRs from each task block, and generate task labels for each timepoint

In [ ]:
# find the condition label for timepoints that are beyond the intial 4 seconds
def get_cond_info(
    layout, deriv_layout, t_r, cleaned_images, blocklen=20, n_trials_to_skip=2
):
    cond_info = {}

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        runs = get_subject_runs(subject, data_dir)
        cond_info[subject] = {}
        for run in runs:
            events_file = layout.get(
                subject=subject,
                run=run,
                datatype='func',
                extension='tsv',
                return_type='file',
            )[0]
            events = pd.read_csv(events_file, sep='\t')
            n_timepoints = cleaned_images[subject][run][1].shape[-1]
            timepoints = np.arange(0, n_timepoints * t_r, t_r)

            # find task onsets in the events file
            # skip 2 trials i.e. 4 seconds
            blocklen = (
                20  # block length in seconds after removing first 4 seconds
            )

            conditions = events.trial_type.unique().tolist()
            conditions.sort()
            onsets = {}
            for condition in conditions:
                match_df = events[events.trial_type == condition]
                onsets[condition] = match_df.onset.tolist()[n_trials_to_skip]
            cond_df = pd.DataFrame(
                {'timepoint': timepoints, 'condition': None}
            )
            for idx in cond_df.index:
                for condition in conditions:
                    if cond_df.loc[idx, 'timepoint'] >= onsets[
                        condition
                    ] and cond_df.loc[idx, 'timepoint'] < (
                        onsets[condition] + blocklen
                    ):
                        cond_df.loc[idx, 'condition'] = condition
            for cond in cond_df.condition.unique():
                if cond is None:
                    continue
                assert len(cond_df[cond_df.condition == cond]) == 8
            cond_info[subject][run] = cond_df
    return cond_info


cond_info = get_cond_info(layout, deriv_layout, t_r, cleaned_images)

In [ ]:


def get_task_images(cond_info, cleaned_images):
    task_images = {}
    task_info = {}
    for subject, runs in cond_info.items():
        task_images[subject] = {}
        task_info[subject] = {}
        for run, cond_df in runs.items():
            good_trials = cond_df.dropna()
            assert (
                len(good_trials) == 64
            ), f'Found {len(good_trials)} good trials for subject {subject} run {run}'
            task_img_file = cleaned_images[subject][run][0].replace(
                'cleaned', 'task'
            )
            assert task_img_file != cleaned_images[subject][run][0]
            good_trials.to_csv(
                task_img_file.replace('_bold.nii.gz', '_events.tsv'),
                sep='\t',
                index=False,
            )
            task_info[subject][run] = good_trials
            if not os.path.exists(task_img_file):
                cleaned_img = cleaned_images[subject][run][1]
                task_data = cleaned_img.get_fdata()[
                    :, :, :, list(good_trials.index)
                ]
                task_img = nib.Nifti1Image(task_data, cleaned_img.affine)
                task_img.to_filename(task_img_file)
            else:
                task_img = nib.load(task_img_file)
            task_images[subject][run] = task_img
    return task_images, task_info


task_images, task_info = get_task_images(cond_info, cleaned_images)

### save to HDF5 file

In [ ]:
subjects = [int(sub) for sub in layout.get_subjects()]


difumo = datasets.fetch_atlas_difumo(
    dimension=1024, resolution_mm=3, legacy_format=False
)
difumo_masker = MultiNiftiMapsMasker(
    maps_img=difumo.maps,
    n_jobs=12,
)

vtmaskdir = os.path.join(data_dir, 'derivatives', 'vtmasks')
with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'w') as hf:

    for subject in subjects:
        print(f'Processing subject {subject}')
        g1 = hf.create_group(f'sub-{subject}')

        runs = get_subject_runs(subject, data_dir)
        sub_mask = get_subject_common_brain_mask(subject, data_dir)
        masker = NiftiMasker(mask_img=sub_mask)
        vt_mask = nib.load(
            os.path.join(
                vtmaskdir,
                f'sub-{subject}_mask4vt_space-MNI152NLin2009cAsym_res-3.nii.gz',
            )
        )

        vtmasker = NiftiMasker(mask_img=vt_mask)
        for run in runs:
            g2 = g1.create_group(f'run-{run}')

            vt_data = vtmasker.fit_transform(task_images[subject][run])
            assert vt_data.shape[0] == task_images[subject][run].shape[-1]
            assert vt_data.shape[1] == np.sum(vt_mask.get_fdata())
            g2.create_dataset('vtmaskdata', data=vt_data)

            # get the whole brain data
            braindata = masker.fit_transform(task_images[subject][run])
            assert braindata.shape[0] == task_images[subject][run].shape[-1]
            assert braindata.shape[1] == np.sum(sub_mask.get_fdata())
            g2.create_dataset('braindata', data=braindata)

            g2.create_dataset(
                'conditions',
                data=[
                    c.encode('utf-8')
                    for c in task_info[subject][run].condition.tolist()
                ],
            )

            # get the difumo data
            difumo_data = difumo_masker.fit_transform(
                task_images[subject][run]
            )
            assert difumo_data.shape[0] == task_images[subject][run].shape[-1]
            g2.create_dataset('difumodata', data=difumo_data)

### Average within blocks



In [ ]:
with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'a') as hf:

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        runs = get_subject_runs(subject, data_dir)
        for run in runs:
            conditions = [
                i.decode('utf-8')
                for i in hf[f'sub-{subject}/run-{run}']['conditions'][:]
            ]

            vt_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['vtmaskdata'][:]
            )
            condition_mean_df = vt_df.groupby(conditions).mean().sort_index()
            conditions_collapsed = condition_mean_df.index.values
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_vtmaskdata', data=condition_mean_df
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_conditions', data=condition_mean_df.index.values
            )

            # same for whole brain
            brain_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['braindata'][:]
            )
            condition_mean_df = (
                brain_df.groupby(conditions).mean().sort_index()
            )
            # make sure conditions are the same between the two
            assert np.all(
                condition_mean_df.index.values == conditions_collapsed
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_braindata', data=condition_mean_df
            )

            # same for difumo
            difumo_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['difumodata'][:]
            )
            condition_mean_df = (
                difumo_df.groupby(conditions).mean().sort_index()
            )
            # make sure conditions are the same between the two
            assert np.all(
                condition_mean_df.index.values == conditions_collapsed
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_difumodata', data=condition_mean_df
            )

In [ ]:
condition_mean_df.sort_index()

In [ ]:

with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'r') as hf:

    # utils.list_all_datasets(hf)
    print(hf[f'sub-{subject}/run-{run}']['braindata'].shape)
    print(hf[f'sub-{subject}/run-{run}']['vtmaskdata'].shape)
    print(hf[f'sub-{subject}/run-{run}']['difumodata'].shape)

    print(hf[f'sub-{subject}/run-{run}']['conditions'].shape)
    print(hf[f'sub-{subject}/run-{run}']['mean_conditions'].shape)